In [1]:
import pandas as pd
import os
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

c:\Users\VSIE43\AppData\Local\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_pth = r'C:\Users\VSIE43\OneDrive - Scania CV\Thesis\SKAB-master\data/'
normal_file = 'anomaly-free/anomaly-free.csv'
test_1 = 'other'
test_2 = 'valve1'
test_3 = 'valve2'

df = pd.read_csv(data_pth+normal_file, sep=';')

df.columns

def fillin_time_gaps(df):
    df.datetime = pd.to_datetime(df.datetime.values)
    time_diff = np.diff(df.datetime.values)

    # there will be need for data imputation. 
    # some samples are with differnce of 2 seconds, rather than 1 second
    new_time = pd.date_range(df.datetime.min(), df.datetime.max(),freq='1s')
    missing_time = pd.DataFrame({'datetime' : new_time})
    df_new = missing_time.merge(df, on='datetime', how='left')

    # maybe fill in with interpolation
    df_new = df_new.interpolate(method='ffill')
    return df_new

normal_data = []
df = pd.read_csv(data_pth+normal_file, sep=';')
normal_data.append(fillin_time_gaps(df).drop(columns=['datetime']))
for folder in [test_2, test_3]:
    files = os.listdir(data_pth+folder)
    print(files)
    for file in files:
        tmp = pd.read_csv(data_pth+folder+'/'+file, sep=';')
        # print(tmp.columns)
        # t = tmp[tmp.anomaly==1]
        t=tmp[tmp.anomaly==0]
        t = t.drop(columns=['changepoint'])
        normal_data.append(t)

df = fillin_time_gaps(df)

['0.csv', '1.csv', '10.csv', '11.csv', '12.csv', '13.csv', '14.csv', '15.csv', '2.csv', '3.csv', '4.csv', '5.csv', '6.csv', '7.csv', '8.csv', '9.csv']
['0.csv', '1.csv', '2.csv', '3.csv']


In [ ]:
# Graph from NTS-No Tears
adj = np.array([[1]+[0]*6+[1], [0]*2+[1]+[0]*5, [0]*2+[1]+[0]*5, [1]+[0]*2+[1]*2+[0]+[1]*2, [0]*4+[1]+[0]*3, [0]*8, [1]+[0]*5+[1]+[0], [0]*6+[1]*2 ])
def adj_to_edge_index(adj):
    edge_index = []
    num_nodes = adj.shape[0]
    for i in range(num_nodes):
        for j in range(num_nodes):
            if adj[i, j] == 1:
                edge_index.append([i, j])
    return torch.tensor(edge_index, dtype=torch.long).t().contiguous()

edge_index = adj_to_edge_index(adj)
edge_index

In [3]:
def min_max_normalize(df, m_m_params=None):
    
    if m_m_params:
        (min_p, max_p) = m_m_params
    else:
        (min_p, max_p) = df.min(), df.max()

    new_df = (df-min_p) / (max_p-min_p)

    return new_df,  (min_p, max_p)


data, m_m_params = min_max_normalize(df.drop(columns=['datetime']))

test_df_1 = pd.read_csv(data_pth+test_1+'/5.csv',sep=';')
test_df_1_norm, _  = min_max_normalize(test_df_1.drop(columns=['datetime','anomaly','changepoint']), m_m_params)

# Enable synchronous error reporting
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Function to create temporal node features
def create_temporal_features(df):
    features = []
    for t in range(len(df)):
        x_t = torch.tensor(df.iloc[t].values, dtype=torch.float).view(-1, 1)
        features.append(x_t)
    return features

# Split the data into training and validation sets
# train_data = data[:int(0.8*len(data))]
# val_data = data[int(0.8*len(data)):]

# # Normalize the validation data using the same min-max parameters
# val_data, _ = min_max_normalize(val_data, m_m_params)

# Create temporal node features for training, validation, and testing
temporal_features_train = create_temporal_features(data)
# temporal_features_val = create_temporal_features(val_data)
temporal_features_test = create_temporal_features(test_df_1_norm)

def create_sliding_windows(features, window_size, stride):
    windows = []
    next_step_t = []

    for i in range(0, len(features) - window_size, stride):
        window = features[i:i + window_size]
        next_step_window = features[i + 1:i + window_size + 1]
        
        if len(next_step_window) == window_size:  # Ensure that the next step window has the same length as the window
            windows.append(window)
            next_step_t.append(next_step_window)

    return np.array(windows), np.array(next_step_t)

# Example usage
window_size = 16  # Define your window size
stride = 1  # Define the stride for the sliding window

# Create sliding windows for training, validation, and testing
temporal_windows_train, temporal_windows_train_next_step = create_sliding_windows(temporal_features_train, window_size, stride)
# temporal_windows_val, temporal_windows_val_next_step = create_sliding_windows(temporal_features_val, window_size, stride)
temporal_windows_test, temporal_windows_test_next_step = create_sliding_windows(temporal_features_test, window_size, stride)

print(f"temporal_windows_train shape: {temporal_windows_train.shape}")
print(f"temporal_windows_train_next_step shape: {temporal_windows_train_next_step.shape}")
print(f"temporal_windows_test shape: {temporal_windows_test.shape}")
print(f"temporal_windows_test_next_step shape: {temporal_windows_test_next_step.shape}")

assert len(temporal_windows_train) == len(temporal_windows_train_next_step), "Mismatch in training windows and next steps"
assert len(temporal_windows_test) == len(temporal_windows_test_next_step), "Mismatch in testing windows and next steps"

class TimeSeriesDataset(Dataset):
    def __init__(self, windows, next_step_t):
        self.windows = windows
        self.next_step_t = next_step_t

    def __len__(self):
        return len(self.windows)

    def __getitem__(self, idx):
        window = self.windows[idx]
        next_step_window = self.next_step_t[idx]
        return window, next_step_window

# Check if CUDA is available and set the device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move edge_index to the GPU
num_nodes = len(data.columns)

# Fully Connected Graph with Self Connections
# edge_index = torch.tensor([(i, j) for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous().to(device)

# Fully Connected Graph without self connections
edge_index = torch.tensor([(i, j) for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous()

# Create dataset objects
train_dataset = TimeSeriesDataset(temporal_windows_train, temporal_windows_train_next_step)
# val_dataset = TimeSeriesDataset(temporal_windows_val, temporal_windows_val_next_step)
test_dataset = TimeSeriesDataset(temporal_windows_test, temporal_windows_test_next_step)

# Create DataLoader objects
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, drop_last= False)

temporal_windows_train shape: (9945, 16, 8, 1)
temporal_windows_train_next_step shape: (9945, 16, 8, 1)
temporal_windows_test shape: (1139, 16, 8, 1)
temporal_windows_test_next_step shape: (1139, 16, 8, 1)


In [4]:


train_skab = np.savez('train_skab', data.to_numpy())
test_skab = np.savez('test_skab', test_df_1_norm.to_numpy())

In [ ]:
class GCN_LSTM(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_nodes):
        super(GCN_LSTM, self).__init__()
        self.gcn1 = GCNConv(in_channels, hidden_channels * 2)
        self.gcn2 = GCNConv(hidden_channels * 2, hidden_channels * 2)  # Second GCN layer
        self.gcn3 = GCNConv(hidden_channels * 2, hidden_channels * 2)
        self.lstm1 = nn.LSTM(hidden_channels * 2, hidden_channels * 2, num_layers=3, batch_first=True)
        self.decoder = nn.Linear(hidden_channels * 2, out_channels)

    def forward(self, x, edge_index):
        batch_size, sequence_length, num_nodes = x.size()
        gcn_outputs = []

        for t in range(sequence_length):
            gcn_output = self.gcn1(x[:, t, :].view(-1, num_nodes), edge_index)
            gcn_output = F.relu(gcn_output)
            gcn_output = self.gcn2(gcn_output, edge_index)  # Pass through the second GCN layer
            gcn_output = F.relu(gcn_output)
            gcn_output = self.gcn3(gcn_output, edge_index)
            gcn_output = F.relu(gcn_output)
            gcn_output = gcn_output.view(batch_size, num_nodes, -1)
            gcn_outputs.append(gcn_output)

        gcn_outputs = torch.stack(gcn_outputs, dim=1)  # Shape: [batch_size, sequence_length, num_nodes, hidden_channels*2]
        gcn_outputs = gcn_outputs.view(batch_size, sequence_length, -1)  # Flatten the node dimension for LSTM

        lstm_output, _ = self.lstm1(gcn_outputs)  # Shape: [batch_size, sequence_length, hidden_channels*2]

        next_step = self.decoder(lstm_output)  # Shape: [batch_size, sequence_length, out_channels]

        return next_step
    
# Initialize the model, optimizer, and other components
in_channels = 8  # One feature per node (the time-series value at each timestep)
hidden_channels = 16
out_channels = 8
num_nodes = len(data.columns)

# model = GCN_LSTM(in_channels=in_channels, hidden_channels=hidden_channels, out_channels=out_channels, num_nodes=num_nodes).to(device)
model = GCN_LSTM(in_channels=in_channels, hidden_channels=hidden_channels, out_channels=out_channels, num_nodes=num_nodes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, amsgrad=True)
print(model)

In [ ]:
def train(train_loader, model, optimizer, num_epochs):
    train_losses = []
    val_losses = []
    all_train_predictions = []
    all_train_targets = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc="Training Batches", leave=False):
            windows, next_step_windows = batch
            # print(windows.size())
            # Move data to the appropriate device (CPU or GPU)
            # windows = windows.to(device)
            # windows = windows
            # next_step_windows = next_step_windows.to(device)
            # next_step_windows = next_step_windows

            optimizer.zero_grad()

            # Remove the extra dimension
            windows = windows.squeeze(-1)
            next_step_windows = next_step_windows.squeeze(-1) 

            # Assuming edge_index is available and already moved to the device
            next_step_pred = model(windows, edge_index)

            # Compute loss
            loss = F.mse_loss(next_step_pred, next_step_windows)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Collect predictions and targets for analysis
            all_train_predictions.append(next_step_pred.detach().cpu().numpy())
            all_train_targets.append(next_step_windows.detach().cpu().numpy())

        avg_train_loss = total_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # # Validation
        # model.eval()
        # val_loss = 0
        # with torch.no_grad():
        #     for val_batch in tqdm(val_loader, desc="Validation Batches", leave=False):
        #         windows, next_step_windows = val_batch

        #         # Move data to the appropriate device (CPU or GPU)
        #         windows = windows.to(device)
        #         next_step_windows = next_step_windows.to(device)

        #         # Remove the extra dimension
        #         windows = windows.squeeze(-1)
        #         next_step_windows = next_step_windows.squeeze(-1)  # Use the last timestep for the target

        #         # Assuming edge_index is available and already moved to the device
        #         next_step_pred = model(windows, edge_index)

        #         # Compute loss
        #         loss = F.mse_loss(next_step_pred, next_step_windows)
        #         val_loss += loss.item()

        # avg_val_loss = val_loss / len(val_loader)
        # val_losses.append(avg_val_loss)

        print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}')

    return train_losses, val_losses, all_train_predictions, all_train_targets

# Train the model
num_epochs = 40
train_losses, val_losses, train_predictions, train_targets = train(train_loader, model, optimizer, num_epochs)

In [ ]:
def inference(test_loader, model):
    model.eval()
    total_loss = 0
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Inference Batches", leave=False):
            windows, next_step_windows = batch


            # Move data to the appropriate device (CPU or GPU)
            # windows = windows.to(device)
            # next_step_windows = next_step_windows.to(device)

            # Remove the extra dimension
            windows = windows.squeeze(-1)
            next_step_windows = next_step_windows.squeeze(-1)


            # Assuming edge_index is available and already moved to the device
            next_step_pred = model(windows, edge_index)

            # Compute loss
            loss = F.mse_loss(next_step_pred, next_step_windows)
            total_loss += loss.item()

            # Collect predictions and targets for analysis
            all_predictions.append(next_step_pred.cpu().numpy())
            all_targets.append(next_step_windows.cpu().numpy())

    # Concatenate all predictions and targets
    all_predictions = np.concatenate(all_predictions, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)

    average_loss = total_loss / len(test_loader)
    return average_loss, all_predictions, all_targets

# Perform inference on the test data
test_loss, test_predictions, test_targets = inference(test_loader, model)
print(f'Test Loss: {test_loss:.4f}')


In [ ]:
actual_values = []
predicted_values = []
model.eval()
with torch.no_grad():
    for sequences, labels in train_loader:
        # sequences = sequences.squeeze(-1).to(device)
        sequences = sequences.squeeze(-1)
        # labels = labels.squeeze(-1).to(device)
        labels = labels.squeeze(-1)
        outputs = model(sequences, edge_index)
        actual_values.extend(labels.cpu().numpy())
        predicted_values.extend(outputs.cpu().numpy())

In [ ]:
residual_train = np.sum((np.array(actual_values)-np.array(predicted_values)), axis=(1))
plt.hist(residual_train)
threshold = np.percentile(residual_train, 90)

In [ ]:
from scipy.stats import iqr
err_median = np.percentile(residual_train, 50,  axis=(0))
err_iqr = iqr(residual_train,axis=(0))

epsilon=1e-2
print((residual_train).shape, err_median.shape, err_iqr.shape)
residual_tmp_train = np.max((residual_train - err_median) / ( np.abs(err_iqr) +epsilon), axis=(-1))

In [ ]:
np.percentile(residual_tmp_train, 90)

In [ ]:
residual_test = np.sqrt(np.sum((test_predictions-test_targets)**2, axis=(1,2)))

plt.plot(residual_test)
plt.plot(test_df_1.anomaly)
# plt.axhline(y = threshold)

In [ ]:
plt.plot(test_df_1.anomaly)

In [ ]:
# Save the predictions and targets for further analysis
np.save('test_predictions.npy', test_predictions)
np.save('test_targets.npy', test_targets)

def plot_losses(train_losses, val_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Losses')
    plt.legend()
    plt.show()

# Plot the losses
plot_losses(train_losses, val_losses)

In [ ]:
from scipy.stats import iqr
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
import pandas as pd
preds = []
labels_tmp = []
file_number = []
# m = model_ext
residual_tmp_all = []

thresholds = [1 , 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50]

def f1_score(true, pred):
    return metrics.f1_score(true, pred)

def recall_score(true, pred):
    return metrics.recall_score(true, pred)

def auc_score(true, pred):
    return metrics.roc_auc_score(true, pred)

def fpr_score(true, pred):
    tn, fp, fn, tp = confusion_matrix(true, pred).ravel()
    return fp / (fp + tn)

# Initialize a dictionary to store the results
results = {
    'f1_score': [],
    'fpr_score': [],
    'recall_score': [],
    'auc_score': []
}

results = {th: {'preds': []} for th in thresholds}

files = os.listdir(data_pth+test_1)
model.eval()
print(files)
for file in files:
    print(file)
    tmp_actual = []
    tmp_pred = []
    tmp = pd.read_csv(data_pth+test_1+'/'+file, sep=';')
    
    t = tmp.anomaly.values
    test_df_1_tmp, _  = min_max_normalize(tmp.drop(columns=['datetime','anomaly','changepoint']),m_m_params)
    temporal_windows_tmp, temporal_windows_tmp_next_step = create_sliding_windows(test_df_1_tmp, window_size = 16, stride = 1)
    # print(temporal_windows_tmp.shape)
    # print(temporal_windows_tmp_next_step.shape)
    tmp_dataset = TimeSeriesDataset(temporal_windows_tmp, temporal_windows_tmp_next_step)
    # Create DataLoader objects
    tmp_loader = DataLoader(tmp_dataset, batch_size=64, shuffle=False)
    with torch.no_grad():
        for sequences, labels in tmp_loader:
            sequences = sequences.squeeze(-1).float()
            print(sequences.shape)
            labels = labels.squeeze(-1).float()
            outputs = model(sequences, edge_index)
            tmp_actual.extend(labels.cpu().numpy())
            tmp_pred.extend(outputs.cpu().numpy())
    residual_tmp = np.sqrt(np.sum((np.array(tmp_pred)-np.array(tmp_actual))**2, axis=(1,2)))

    residual_tmp_all.append(residual_tmp)
    labels_tmp.append(t[-len(tmp_pred):])
    i = int(file.split('.')[0])
    file_number.append([i]*len(tmp_pred))
    # for th in thresholds:
    #     results[th]['preds'].append(residual_tmp > th)
    i +=1
    # break
# for th in thresholds:
#     results[th]['preds'] = np.concatenate(results[th]['preds'], axis=0)
preds = np.concatenate(preds, axis=0)
labels = np.concatenate(labels_tmp, axis=0)
file_number = np.concatenate(file_number, axis=0)


In [ ]:
tmp_anom_list = []
for file in files:
    tmp_df = pd.read_csv(data_pth+test_1+'/'+file, sep=';')
    tmp_anom_list.append(tmp_df.anomaly)
    plt.plot(tmp_df.anomaly)

In [ ]:
len(tmp_anom_list)

In [ ]:
plt.plot(residual_tmp)


In [ ]:
residual_tmp_all = np.concatenate(residual_tmp_all)

In [ ]:
from scipy.stats import iqr

for i in range(len(residual_tmp_all)):
    err_median = np.median(residual_tmp_all[i])
    err_iqr = iqr(residual_tmp_all[i])

    epsilon=1e-2

    residual_tmp_all[i] = (residual_tmp_all[i] - err_median) / ( np.abs(err_iqr) +epsilon)

In [ ]:
files

In [ ]:
plt.plot(residual_tmp)

In [ ]:
for i in range(len(residual_tmp_all)):
    fig, ax1 = plt.subplots()  # Create a new figure and axis for residuals

    ax1.plot(residual_tmp_all[i], 'b-')
    ax1.set_title(f'Residual for File {i+1}')
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Residuals', color='b')

    ax2 = ax1.twinx()  # Create a second y-axis
    ax2.plot(tmp_anom_list[i], 'r-')
    ax2.set_ylabel('Anomaly', color='r')
    ax2.set_ylim(-0.1, 1.1)  # Set limits for the anomaly axis

    plt.show()

In [ ]:
preds

In [ ]:
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
import pandas as pd



# Define evaluation metrics functions
def f1_score(true, pred):
    return metrics.f1_score(true, pred)

def recall_score(true, pred):
    return metrics.recall_score(true, pred)

def auc_score(true, pred):
    return metrics.roc_auc_score(true, pred)

def fpr_score(true, pred):
    tn, fp, fn, tp = confusion_matrix(true, pred).ravel()
    return fp / (fp + tn)

# Initialize a dictionary to store the results
results = {
    'f1_score': [],
    'fpr_score': [],
    'recall_score': [],
    'auc_score': []
}

# Loop through the files and calculate the metrics
for f in range(1, 15):
    true = labels[file_number == f]
    pred = preds.astype(int)[file_number == f]
    
    results['f1_score'].append(f1_score(true, pred))
    results['fpr_score'].append(fpr_score(true, pred))
    results['recall_score'].append(recall_score(true, pred))
    results['auc_score'].append(auc_score(true, pred))
print(results)
# Create a DataFrame from the results dictionary
df = pd.DataFrame(results)

# Define the file path to save the results
file_path = 'graphvs_ae_anomaly_detection_scores.xlsx'

# Save the DataFrame to a new sheet in the existing Excel file
with pd.ExcelWriter(file_path, mode='a', engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Additional Scores', index=False)

print("Results have been saved to a new sheet in", file_path)


In [ ]:
results

In [ ]:
metrics_results = {th: {'f1_score': [], 'fpr_score': [], 'recall_score': [], 'auc_score': []} for th in thresholds}

# Loop through the thresholds to calculate the metrics for each file
for th in thresholds:
    # The preds are now directly accessed from the 'results' dictionary
    for f in range(1, 15):  # Adjust range as per your actual file indices
        true = labels[file_number == f]
        pred = results[th]['preds'].astype(int)[file_number == f]
        # print(len(pred))
        metrics_results[th]['f1_score'].append(f1_score(true, pred))
        metrics_results[th]['fpr_score'].append(fpr_score(true, pred))
        metrics_results[th]['recall_score'].append(recall_score(true, pred))
        metrics_results[th]['auc_score'].append(auc_score(true, pred))

# If you need to print or analyze aggregated results:
for th in thresholds:
    print(f"Threshold {th}:")
    print(f"F1 Score: {np.mean(metrics_results[th]['f1_score'])}")
    print(f"FPR Score: {np.mean(metrics_results[th]['fpr_score'])}")
    print(f"Recall Score: {np.mean(metrics_results[th]['recall_score'])}")
    print(f"AUC Score: {np.mean(metrics_results[th]['auc_score'])}")
    print("\n")

data_frames = []
for th, scores in metrics_results.items():
    # Calculate the mean of each list of scores for the current threshold
    mean_scores = {metric: np.mean(values) for metric, values in scores.items()}
    mean_scores['Threshold'] = th
    # Convert mean_scores to a DataFrame and append to data_frames list
    df = pd.DataFrame([mean_scores])  # Create a DataFrame with a single row of mean_scores
    data_frames.append(df)

# Concatenate all data frames into a single frame
final_df = pd.concat(data_frames, ignore_index=True)

# File path to your Excel file
file_path = 'graphvs_ae_anomaly_detection_scores.xlsx'

# Load the existing Excel file
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    # Check if the file already exists; if not, write a new file
    try:
        _ = pd.read_excel(file_path)
    except FileNotFoundError:
        # If the file does not exist, create it and write the DataFrame
        final_df.to_excel(writer, index=False, sheet_name='Anomaly Detection Scores')
    else:
        # Write to a new sheet in the existing file
        final_df.to_excel(writer, index=False, sheet_name='Anomaly Detection Scores')

print("Data has been written to the Excel file.")

In [ ]:
file_number